# Task02 Pandas基础
## 1 知识梳理（重点记忆）

### 1.1 文件的读取与写入

通过使用`parse_datas`参数，将日期进行格式化

In [1]:
import pandas as pd

pd.read_csv('../data/my_csv.csv', parse_dates=['col5'])

,col1,col2,col3,col4,col5
0,2,a,1.4,apple,2020-01-01
1,3,b,3.4,banana,2020-01-02
2,6,c,2.5,orange,2020-01-05
3,5,d,3.2,lemon,2020-01-07


### 1.2 常用基本函数

#### 1.2.1 `info`和`describe`函数

In [2]:
df = pd.read_csv('../data/learn_pandas.csv')
df.columns

Index(['School', 'Grade', 'Name', 'Gender', 'Height', 'Weight', 'Transfer',
       'Test_Number', 'Test_Date', 'Time_Record'],
      dtype='object')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   School       200 non-null    object 
 1   Grade        200 non-null    object 
 2   Name         200 non-null    object 
 3   Gender       200 non-null    object 
 4   Height       183 non-null    float64
 5   Weight       189 non-null    float64
 6   Transfer     188 non-null    object 
 7   Test_Number  200 non-null    int64  
 8   Test_Date    200 non-null    object 
 9   Time_Record  200 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 15.8+ KB


主要展示数据集里面的列名、非空个数、对应的数据类型，统计列类型的个数和数据集占用的内存

In [4]:
df.describe()

,Height,Weight,Test_Number
count,183.000000,189.000000,200.000000
mean,163.218033,55.015873,1.645000
std,8.608879,12.824294,0.722207
min,145.400000,34.000000,1.000000
25%,157.150000,46.000000,1.000000
50%,161.900000,51.000000,1.500000
75%,167.500000,65.000000,2.000000
max,193.900000,89.000000,3.000000


&emsp;&emsp;主要展示数据集中类型为`float`和`int`的统计值，包括个数、均值、标准差、最小值、25%分位数、50%分位数（中位数）、75%分位数和最大值

#### 1.2.2 `drop_duplicates`函数
&emsp;&emsp;`drop_duplicates`用途是对重复数据进行数据清理。   
&emsp;&emsp;如果想要观察多个列组合的唯一值，可以使用`drop_duplicates`。其中的关键参数是`keep`，默认值`first`表示每个组合保留第一次出现的所在行，`last`表示保留最后一次出现的所在行，`False`表示把所有重复组合所在的行剔除。

In [5]:
df_demo = df[['Gender','Transfer','Name']]
df_demo.drop_duplicates(['Gender', 'Transfer'])

,Gender,Transfer,Name
0,Female,N,Gaopeng Yang
1,Male,N,Changqiang You
12,Female,NaN,Peng You
21,Male,NaN,Xiaopeng Shen
36,Male,Y,Xiaojuan Qin
43,Female,Y,Gaoli Feng


#### 1.2.3 `replace`函数
`replace`函数可以进行方向替换，指定`method`参数为`ffill`则为用前面一个最近的未被替换的值进行替换，`bfill`则使用后面最近的未被替换的值进行替换。

In [6]:
s = pd.Series(['a', 1, 'b', 2, 1, 1, 'a'])
s.replace([1, 2], method='ffill')

0    a
1    a
2    b
3    b
4    b
5    b
6    a
dtype: object

#### 1.2.4 `where`和`mask`函数  
`where`函数是根据条件进行反向过滤，`mask`函数是根据条件进行正向过滤。

In [7]:
s = pd.Series([-1, 1.2345, 100, -50])
s.where(s<0, 100)

0     -1.0
1    100.0
2    100.0
3    -50.0
dtype: float64

In [8]:
s.mask(s<0, -100)

0   -100.0000
1      1.2345
2    100.0000
3   -100.0000
dtype: float64

#### 1.2.5 窗口对象
1. 滑窗对象：`rolling`函数，可使用`window`参数设置滑动窗口

In [9]:
s = pd.Series([1,2,3,4,5])
roller = s.rolling(window = 3)
roller.mean()

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
dtype: float64

2. 扩张窗口：又称为累计窗口，`expanding`函数

In [10]:
s.expanding().sum()

0     1.0
1     3.0
2     6.0
3    10.0
4    15.0
dtype: float64

## 2 练一练

### 2.1 第1题
在`clip`中，超过边界的只能截断为边界值，如果要把超出边界的替换为自定义的值，应当如何做？

In [11]:
import pandas as pd
s = pd.Series([-1, 1.2345, 100, -50])

**我的解答：**  
根据题意理解，假设要替换的值为(-2, 30)，其中下边界为-2、上边界为30，故得到的序列应该为`[-2, 1.2345, 30, -2]`

In [12]:
def replace_clip(s, lower, upper, define_lower_value, define_upper_value):
    return s.mask(s<lower, define_lower_value).mask(s>upper, define_upper_value)

In [13]:
replace_clip(s, lower=0, upper=2, define_lower_value=-2, define_upper_value=30)

0    -2.0000
1     1.2345
2    30.0000
3    -2.0000
dtype: float64

### 2.2 第2题
`rolling`对象的默认窗口方向都是向前的，某些情况下用户需要向后的窗口，例如对1,2,3设定向后窗口为2的`sum`操作，结果为3,5,NaN，此时应该如何实现向后的滑窗操作？（提示：使用`shift`）

**我的解答：**  
例如对`[1,2,3,4,5]`设定向后窗口为3的`sum`操作，结果应为`[6,9,12,NaN,NaN]`

In [14]:
s = pd.Series([1, 2, 3, 4, 5])
s.rolling(3).sum().shift(-2)

0     6.0
1     9.0
2    12.0
3     NaN
4     NaN
dtype: float64

### 2.3 第3题
`cummax`, `cumsum`, `cumprod`函数是典型的类扩张窗口函数，请使用`expanding`对象依次实现它们。

**我的解答：**

In [15]:
s = pd.Series([5,4,3,2,1])

1. **累计求最大值**

函数`cummax`的执行结果：

In [16]:
s.cummax()

0    5
1    5
2    5
3    5
4    5
dtype: int64

使用`expanding`实现的`cummax`：

In [17]:
s.expanding().max()

0    5.0
1    5.0
2    5.0
3    5.0
4    5.0
dtype: float64

2. **累计求和**

函数`cumsum`的执行结果：

In [18]:
s.cumsum()

0     5
1     9
2    12
3    14
4    15
dtype: int64

使用`expanding`实现的`cumsum`：

In [19]:
s.expanding().sum()

0     5.0
1     9.0
2    12.0
3    14.0
4    15.0
dtype: float64

3. **累计求积**

函数`cumprod`的执行结果：

In [20]:
s.cumprod()

0      5
1     20
2     60
3    120
4    120
dtype: int64

使用`expanding`实现的`cumprod`：

In [21]:
import numpy as np
s.expanding().apply(lambda x: np.array(x).prod())

0      5.0
1     20.0
2     60.0
3    120.0
4    120.0
dtype: float64

## 3 练习

### 3.1 Ex1：口袋妖怪数据集
现有一份口袋妖怪的数据集，下面进行一些背景说明：

* `#`代表全国图鉴编号，不同行存在相同数字则表示为该妖怪的不同状态

* 妖怪具有单属性和双属性两种，对于单属性的妖怪，`Type 2`为缺失值
* `Total, HP, Attack, Defense, Sp.Atk, Sp.Def, Speed`分别代表种族值、体力、物攻、防御、特攻、特防、速度，其中种族值为后6项之和

In [22]:
df = pd.read_csv('../data/pokemon.csv')
df.head(3)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80


1. 对`HP, Attack, Defense, Sp. Atk, Sp. Def, Speed`进行加总，验证是否为`Total`值。

2. 对于`#`重复的妖怪只保留第一条记录，解决以下问题：

* 求第一属性的种类数量和前三多数量对应的种类
* 求第一属性和第二属性的组合种类
* 求尚未出现过的属性组合

3. 按照下述要求，构造`Series`：

* 取出物攻，超过120的替换为`high`，不足50的替换为`low`，否则设为`mid`
* 取出第一属性，分别用`replace`和`apply`替换所有字母为大写
* 求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到`df`并从大到小排序

**我的解答：**  
**第1问：**

In [23]:
df = pd.read_csv('../data/pokemon.csv')
df[df['Total'] != df[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']].sum(1)]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed


通过观察，可知所有数据都是的6个属性的加总与`Total`值是一致的。

**第2问：**

In [24]:
# 对'#'重复的妖怪只保留第一条记录
df_dropdup = df.drop_duplicates('#', keep='first')
df_dropdup.shape

(721, 11)

In [25]:
# 第一属性的种类数量
df_dropdup['Type 1'].nunique()

18

In [26]:
# 前三多数量对应的种类
df_dropdup['Type 1'].value_counts().head(3)

Water     105
Normal     93
Grass      66
Name: Type 1, dtype: int64

In [27]:
# 第一属性和第二属性的组合种类
df_type1_type2 = df_dropdup.drop_duplicates(['Type 1', 'Type 2'])
df_type1_type2[['Type 1', 'Type 2']]

,Type 1,Type 2
0,Grass,Poison
4,Fire,NaN
6,Fire,Flying
9,Water,NaN
13,Bug,NaN
...,...,...
773,Rock,Fairy
778,Ghost,Grass
790,Flying,Dragon
797,Psychic,Ghost


In [28]:
# 求尚未出现过的属性组合
type1_list = df_dropdup['Type 1'].unique()

In [29]:
type2_list = df_dropdup['Type 2'].unique()
# 删除nan的数据
type2_list = type2_list[~pd.isnull(type2_list)]

In [30]:
import numpy as np
set_full = set([(i,j) if i!=j else None for i in type1_list for j in type2_list])
len(set_full)

307

In [31]:
set_used = set((i, j) if type(j) == str else None for i,j in zip(df_type1_type2['Type 1'], df_type1_type2['Type 2']))
len(set_used)

126

In [32]:
res = set_full.difference(set_used)
len(res)

181

**第3问：**

In [33]:
# 取出物攻，超过120的替换为high，不足50的替换为low，否则设为mid
df_attack = df['Attack'].copy()

def cond_replace(x):
     if x < 50:
        return 'low'
     elif x > 120:
        return 'high'
     return 'mid'
res = df_attack.apply(cond_replace)
res.head()

0    low
1    mid
2    mid
3    mid
4    mid
Name: Attack, dtype: object

In [34]:
# 取出第一属性，分别用replace和apply替换所有字母为大写
# 使用replace实现
df_type1 = df['Type 1'].copy()
df_type1.replace(df_type1.unique(), [str.upper(s) for s in df_type1.unique()]).head()

0    GRASS
1    GRASS
2    GRASS
3    GRASS
4     FIRE
Name: Type 1, dtype: object

In [35]:
# 使用apply实现
df_type1.apply(lambda x: str.upper(x)).head()

0    GRASS
1    GRASS
2    GRASS
3    GRASS
4     FIRE
Name: Type 1, dtype: object

In [36]:
# 求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到df并从大到小排序
df['Capacity'] = df[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']].apply(lambda x: np.max((x-x.mean()).abs()), 1)
df.sort_values('Capacity',ascending=False)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Capacity
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,175.000000
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,165.000000
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,145.833333
224,208,SteelixMega Steelix,Steel,Ground,610,75,125,230,55,95,30,128.333333
333,306,AggronMega Aggron,Steel,NaN,630,70,140,230,60,80,50,125.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
255,236,Tyrogue,Fighting,NaN,210,35,35,35,35,35,35,0.000000
383,351,Castform,Normal,NaN,420,70,70,70,70,70,70,0.000000
358,327,Spinda,Normal,NaN,360,60,60,60,60,60,60,0.000000
550,492,ShayminLand Forme,Grass,NaN,600,100,100,100,100,100,100,0.000000


### 3.2 Ex2：指数加权窗口
1. 作为扩张窗口的`ewm`窗口

在扩张窗口中，用户可以使用各类函数进行历史的累计指标统计，但这些内置的统计函数往往把窗口中的所有元素赋予了同样的权重。事实上，可以给出不同的权重来赋给窗口中的元素，指数加权窗口就是这样一种特殊的扩张窗口。

其中，最重要的参数是`alpha`，它决定了默认情况下的窗口权重为$w_i=(1−\alpha)^i,i\in\{0,1,...,t\}$，其中$i=t$表示当前元素，$i=0$表示序列的第一个元素。

从权重公式可以看出，离开当前值越远则权重越小，若记原序列为$x$，更新后的当前元素为$y_t$，此时通过加权公式归一化后可知：

$$
\begin{aligned}\
y_t &=\frac{\sum_{i=0}^{t} w_i x_{t-i}}{\sum_{i=0}^{t} w_i} \\
&=\frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^{t} x_{0}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^{t-1}}
\end{aligned}
$$

对于`Series`而言，可以用`ewm`对象如下计算指数平滑后的序列：

In [37]:
np.random.seed(0)
s = pd.Series(np.random.randint(-1,2,30).cumsum())
s.head()

0   -1
1   -1
2   -2
3   -2
4   -2
dtype: int32

In [38]:
s.ewm(alpha=0.2).mean().head()

0   -1.000000
1   -1.000000
2   -1.409836
3   -1.609756
4   -1.725845
dtype: float64

请用`expanding`窗口实现。

2. 作为滑动窗口的`ewm`窗口

从第1问中可以看到，`ewm`作为一种扩张窗口的特例，只能从序列的第一个元素开始加权。现在希望给定一个限制窗口`n`，只对包含自身最近的`n`个窗口进行滑动加权平滑。请根据滑窗函数，给出新的`wi`与`yt`的更新公式，并通过`rolling`窗口实现这一功能。

**我的解答：**  

**第1问：**  
看到$\displaystyle \sum_{i=0}^{t} w_i x_{t-i}$，可想到用`numpy`的乘法，需要构造$w$和$x$矩阵：  
易知$x$矩阵只需要逆转即可，而$w$矩阵只需使用`(1 - alpha) ** np.arange(t)`，其中$t$表示$x$矩阵的长度

In [39]:
def my_ewm(x, alpha):
    # 将x逆转
    x = np.array(x)[::-1]
    # 构造w
    w = (1 - alpha) ** np.arange(len(x))
    return (w * x).sum() / w.sum()

alpha = 0.2
s.expanding().apply(lambda x: my_ewm(x, alpha)).head()

0   -1.000000
1   -1.000000
2   -1.409836
3   -1.609756
4   -1.725845
dtype: float64

**第2问：**  
根据题意，限制窗口为`n`  
$w_i$的更新公式：$w_i=(1−\alpha)^i,i\in\{0,1,...,n-1\}$  
$y_t$的更新公式：
$$
\begin{aligned}\
y_t &=\frac{\sum_{i=0}^{n-1} w_i x_{t-i}}{\sum_{i=0}^{n-1} w_i} \\
&=\frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^{n-1} x_{t-(n-1)}}
{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^{n-1}}
\end{aligned}
$$

In [40]:
def my_ewm(x, alpha):
    x = np.array(x)[::-1]
    w = (1 - alpha) ** np.arange(len(x))
    return (w * x).sum() / w.sum()

alpha = 0.2
s.rolling(window=3).apply(lambda x: my_ewm(x, alpha)).head()

0         NaN
1         NaN
2   -1.409836
3   -1.737705
4   -2.000000
dtype: float64